This is a competation in https://www.drivendata.org/competitions/44/dengai-predicting-disease-spread/page/80/.
Lets understand the dataset for training, testing and sample submission first.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [44]:
train_data = pd.read_csv("data/dengue_features_train.csv", parse_dates=[1,2,3], infer_datetime_format=True)
test_data = pd.read_csv("data/dengue_features_test.csv", parse_dates=[1,2,3], infer_datetime_format=True)
train_lables = pd.read_csv("data/dengue_labels_train.csv", parse_dates=[1,2,3], infer_datetime_format=True)
sample_submission = pd.read_csv("data/submission_format.csv", parse_dates=[1,2,3], infer_datetime_format=True)

Lets understand the data size and its contents in train_data and train

In [45]:
#shape of everything
print("Shape of train data is ", train_data.shape)
print("Shape of test data is ", test_data.shape)
print("Shape of Training Lables data is ", train_lables.shape)
print("sample_submission shape is ", sample_submission.shape)

Shape of train data is  (1456, 24)
Shape of test data is  (416, 24)
Shape of Training Lables data is  (1456, 4)
sample_submission shape is  (416, 4)


This means we have 24 features and 1456 rows of data.Lets look at the sample now.

In [46]:
train_data.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990-01-01,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990-01-01,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990-01-01,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990-01-01,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990-01-01,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [47]:
test_data.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008-01-01,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008-01-01,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008-01-01,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008-01-01,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008-01-01,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


In [48]:
train_lables.head()

,city,year,weekofyear,total_cases
0,sj,1990-01-01,18,4
1,sj,1990-01-01,19,5
2,sj,1990-01-01,20,4
3,sj,1990-01-01,21,3
4,sj,1990-01-01,22,6


Looks like we have city,year,weekofYear as the key for both training labels and training data. Lets look at the dataType of each set.

In [49]:
train_data.dtypes

city                                             object
year                                     datetime64[ns]
weekofyear                                       object
week_start_date                          datetime64[ns]
ndvi_ne                                         float64
ndvi_nw                                         float64
ndvi_se                                         float64
ndvi_sw                                         float64
precipitation_amt_mm                            float64
reanalysis_air_temp_k                           float64
reanalysis_avg_temp_k                           float64
reanalysis_dew_point_temp_k                     float64
reanalysis_max_air_temp_k                       float64
reanalysis_min_air_temp_k                       float64
reanalysis_precip_amt_kg_per_m2                 float64
reanalysis_relative_humidity_percent            float64
reanalysis_sat_precip_amt_mm                    float64
reanalysis_specific_humidity_g_per_kg           

In [50]:
def convert_weekofyear_to_day(date):
    return pd.Period(date, freq='D').dayofyear


In [51]:
train_data['week_start_date'] = train_data['week_start_date'].map(convert_weekofyear_to_day)

In [55]:
train_data.isnull().sum()

city                                       0
year                                       0
weekofyear                                 0
week_start_date                            0
ndvi_ne                                  194
ndvi_nw                                   52
ndvi_se                                   22
ndvi_sw                                   22
precipitation_amt_mm                      13
reanalysis_air_temp_k                     10
reanalysis_avg_temp_k                     10
reanalysis_dew_point_temp_k               10
reanalysis_max_air_temp_k                 10
reanalysis_min_air_temp_k                 10
reanalysis_precip_amt_kg_per_m2           10
reanalysis_relative_humidity_percent      10
reanalysis_sat_precip_amt_mm              13
reanalysis_specific_humidity_g_per_kg     10
reanalysis_tdtr_k                         10
station_avg_temp_c                        43
station_diur_temp_rng_c                   43
station_max_temp_c                        20
station_mi